### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup #클래스 import

In [ ]:
# 식별자 컨벤션 : 변수, 함수(snake_case) : 클래스(PascalCase, uppercamelcase)

#### 1. 웹페이지 분석 : URL

In [13]:
query = "삼성전자"
url = f"https://search.naver.com/search.naver?query={query}"
url

'https://search.naver.com/search.naver?query=삼성전자'

#### 2. request(URL) > response : str(html)    vs 동적은 json
#### 정적은 파싱하는 방법이 더 복잡하다

In [14]:
response = requests.get(url)
response

<Response [200]>

#### 3. str(html) > bs object

In [15]:
dom = BeautifulSoup(response.text, "html.parser")
dom

<!DOCTYPE html>
 <html lang="ko"> <head> <meta charset="utf-8"/> <meta content="always" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="width=device-width,initial-scale=1.0,maximum-scale=2.0" name="viewport"/> <meta content="삼성전자 : 네이버 통합검색" property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'삼성전자'의 네이버 통합검색 결과입니다." property="og:description"/> <meta content="'삼성전자'의 네이버 통합검색 결과입니다." lang="ko" name="description"/> <title>삼성전자 : 네이버 통합검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_230216.css" rel="stylesheet" type="text/css"/> <link href="https://ssl.pstatic.net/sst

#### 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)
#### copy -> copy selector

In [16]:
elements = dom.select("#nx_right_related_keywords > div > div.related_srch > ul > li")
len(elements)
#nx_right_related_keywords > div > div.related_srch > ul > li

10

In [17]:
element = elements[0]
element.select_one(".tit").text

'삼성전자주가'

#### 5. str(text) > DataFrame

In [18]:
keywords = [element.select_one(".tit").text for element in elements]
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자 성과급',
 '삼성전자 주식',
 '삼성전자서비스',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자 세일 페스타',
 '삼성 전자레인지',
 '삼성전자 opi']

In [19]:
df = pd.DataFrame({"keyword" : keywords})
df["query"] = query
df

,keyword,query
0,삼성전자주가,삼성전자
1,삼성전자 배당금,삼성전자
2,삼성전자 성과급,삼성전자
3,삼성전자 주식,삼성전자
4,삼성전자서비스,삼성전자
5,오늘 삼성전자 주가,삼성전자
6,삼성전자 배당금 지급일,삼성전자
7,삼성전자 세일 페스타,삼성전자
8,삼성 전자레인지,삼성전자
9,삼성전자 opi,삼성전자


In [22]:
#현재 시간 데이터 추가
from datetime import datetime
now = datetime.now()
now = now.strftime("%Y-%m-%d %H : %M")
df["date_time"] = now
df.tail(2)

,keyword,query,date_time
8,삼성 전자레인지,삼성전자,2023-02-19 23 : 56
9,삼성전자 opi,삼성전자,2023-02-19 23 : 56


In [26]:
#query를 입력하면 데이터 프레임을 출력하는 함수
def naver_relate_keyword(query):
    url = f"https://search.naver.com/search.naver?query={query}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(".lst_related_srch > .item")
    keywords = [element.text.strip() for element in elements]
    df = pd.DataFrame({"keywords" : keywords})
    df["query"] = query
    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H : %M")
    df["date_time"] = now
    return df

query = "삼성전자"
df = naver_relate_keyword(query)
df.tail(2)

,keywords,query,date_time
8,삼성 전자레인지,삼성전자,2023-02-20 00 : 01
9,삼성전자 opi,삼성전자,2023-02-20 00 : 01


In [27]:
dfs = []
queries = ["삼성전자", "LG전자"]
for query in queries:
    print(query, end = " ")
    df = naver_relate_keyword(query)
    dfs.append(df)
result = pd.concat(dfs, ignore_index = True)
result

삼성전자 LG전자 

,keywords,query,date_time
0,삼성전자주가,삼성전자,2023-02-20 00 : 03
1,삼성전자 배당금,삼성전자,2023-02-20 00 : 03
2,삼성전자 성과급,삼성전자,2023-02-20 00 : 03
3,삼성전자 주식,삼성전자,2023-02-20 00 : 03
4,삼성전자서비스,삼성전자,2023-02-20 00 : 03
5,오늘 삼성전자 주가,삼성전자,2023-02-20 00 : 03
6,삼성전자 배당금 지급일,삼성전자,2023-02-20 00 : 03
7,삼성전자 세일 페스타,삼성전자,2023-02-20 00 : 03
8,삼성 전자레인지,삼성전자,2023-02-20 00 : 03
9,삼성전자 opi,삼성전자,2023-02-20 00 : 03
